### Lecture 1 from CUDA MODE - cuda profilers

In [1]:
import torch

In [2]:
a = torch.tensor([1.,2.,3.])

### Time a pytorch function on the GPU

It is not possible to use the `time` python module because CUDA is Async. \
We can write a function that will use `torch.cuda.Event` to appropriately measure time. There is a warmup time, so the first iteration won't be as fast as the later ones. If we want a measure of the steady state, we have to function we want to measure a few time befaor staring the measure.

In [33]:
def time_pytorch_function(func, input):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)

    #warmup
    for _ in range(5):
        func(input)

    start.record()
    func(input)
    end.record()
    torch.cuda.synchronize()
    return start.elapsed_time(end)

In [34]:
b = torch.randn(10000, 10000).cuda()

In [35]:
def square_2(a):
    return a*a

In [36]:
def square_3(a):
    return a**2

In [37]:
time_pytorch_function(torch.square, b)
time_pytorch_function(square_2, b)
time_pytorch_function(square_3, b)

0.9462720155715942

### We can use `torch.autograd.profiler`

In [38]:
print("=============")
print("Profiling torch.square")
print("=============")

Profiling torch.square


In [39]:
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    torch.square(b)

STAGE:2024-02-20 13:33:29 513:513 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-02-20 13:33:29 513:513 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-02-20 13:33:29 513:513 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [40]:
print("=============")
print("Profiling torch.square")
print("=============")
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

Profiling torch.square
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
             aten::square         0.81%      11.000us         6.28%      85.000us      85.000us       4.000us         0.44%     913.000us     913.000us             1  
                aten::pow         4.06%      55.000us         5.39%      73.000us      73.000us     907.000us        99.34%     909.000us     909.000us             1  
        aten::result_type         0.07%       1.000us         0.07%       1.000us       1.000us       1.000us         0.11%       1.000u

In [41]:
print("=============")
print("Profiling a * a")
print("=============")

Profiling a * a


In [42]:
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    square_2(b)

STAGE:2024-02-20 13:33:29 513:513 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-02-20 13:33:29 513:513 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-02-20 13:33:29 513:513 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [43]:
print("=============")
print("Profiling a * a")
print("=============")
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

Profiling a * a
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                aten::mul         4.41%      54.000us         6.21%      76.000us      76.000us     846.000us       100.00%     846.000us     846.000us             1  
          cudaEventRecord         1.06%      13.000us         1.06%      13.000us       6.500us       0.000us         0.00%       0.000us       0.000us             2  
         cudaLaunchKernel         1.80%      22.000us         1.80%      22.000us      22.000us       0.000us         0.00%       0.000us      

In [44]:
print("=============")
print("Profiling a ** 2")
print("=============")

Profiling a ** 2


In [45]:
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    square_3(b)

STAGE:2024-02-20 13:33:30 513:513 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-02-20 13:33:30 513:513 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-02-20 13:33:30 513:513 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [46]:
print("=============")
print("Profiling a ** 2")
print("=============")
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

Profiling a ** 2
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                aten::pow         6.49%      83.000us         9.00%     115.000us     115.000us     839.000us        99.76%     841.000us     841.000us             1  
        aten::result_type         0.16%       2.000us         0.16%       2.000us       2.000us       1.000us         0.12%       1.000us       1.000us             1  
                 aten::to         0.08%       1.000us         0.08%       1.000us       1.000us       1.000us         0.12%       1.000us     

Running the same code in a python script yields shorter execution times. The iPython environment and the iterative cell mechanics might influence the cuda Events.

### We can use the Pytorch profiler

In [47]:
import torch
from torch.profiler import profile, record_function, ProfilerActivity

We can save the report in JSON format that can be read by Chrome or Brave tracing, drag and drop in the browser at `chrome://tracing/`.

In [52]:
def trace_handler(prof):
    print(prof.key_averages().table(
        sort_by="self_cuda_time_total", row_limit=-1))
    prof.export_chrome_trace("./log/test_trace_" + str(prof.step_num) + ".json")

In [53]:
with torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA,
    ],
     schedule=torch.profiler.schedule(
        wait=1,
        warmup=1,
        active=2,
        repeat=1),
    on_trace_ready=trace_handler
    # on_trace_ready=torch.profiler.tensorboard_trace_handler('./log')
    # used when outputting for tensorboard
    ) as p:
        for iter in range(10):
            torch.square(torch.randn(10000, 10000).cuda())
            # send a signal to the profiler that the next iteration has started
            p.step()

STAGE:2024-02-20 18:20:09 513:513 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-02-20 18:20:10 513:513 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-02-20 18:20:10 513:513 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
            ProfilerStep*         0.11%     999.000us        93.85%     841.049ms     420.524ms             2  
              aten::randn         0.00%      24.000us        52.64%     471.796ms     235.898ms             2  
              aten::empty         0.00%      19.000us         0.00%      19.000us       9.500us             2  
            aten::normal_        52.64%     471.753ms        52.64%     471.753ms     235.876ms             2  
                 aten::to         0.00%      26.000us        41.08%     368.126ms      92.031ms             4  
           aten::_to_copy         0.00%      23.000us        41.07%     368.100ms     184.050ms         